---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);

### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [ ]:
def answer_one():
    
    # Your Code Here
    g = nx.read_edgelist('Employee_Movie_Choices.txt', delimiter = '\t')
    
    return g # Your Answer Here

# G = answer_one()
# G.number_of_nodes()
# G.number_of_edges()
# plot_graph(G)

### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [ ]:
def answer_two():
    
    # Your Code Here
    g = answer_one()
    for node in g.nodes():
        if node in employees:
            g.add_node(node, type='employee')
        else:
            g.add_node(node, type='movie')
    
    return g # Your Answer Here

# G = answer_two()
# G.nodes(data=True)

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [ ]:
def answer_three():
        
    # Your Code Here
    g = answer_two()
    p = bipartite.weighted_projected_graph(g, employees) 
    
    return p # Your Answer Here

# Plot weighted graph
# pos = nx.spring_layout(P)
# edge_weights = {(u,v,):d['weight'] for u,v,d in P.edges(data=True)}
# nx.draw(P, pos = pos, with_labels=True)
# nx.draw_networkx_edge_labels(P, pos,edge_labels=edge_weights)

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [ ]:
def answer_four():
        
    # Your Code Here
    g = answer_three()
    g1 = nx.read_edgelist('Employee_Relationships.txt', delimiter = '\t', data=[('rs_score', int)])

    df_g1 = pd.DataFrame(g1.edges(data=True), columns=['emp1', 'emp2', 'rs_score'])
    df_g1['rs_score'] = df_g1['rs_score'].map(lambda x: x['rs_score'])

    df_g = pd.DataFrame(g.edges(data=True), columns=['emp1', 'emp2', 'common_movies'])
    df_g['common_movies'] = df_g['common_movies'].map(lambda x: x['weight'])

    merge_1 = pd.merge(df_g1, df_g, how='left', on=['emp1', 'emp2'])
    merge_1.rename(columns={'emp1': 'emp2', 'emp2': 'emp1'}, inplace=True)
    merge_2 = pd.merge(merge_1, df_g, how='left', on=['emp2', 'emp1'])

    merge_2['common_movies_x'] = merge_2['common_movies_x'].fillna(0)
    merge_2['common_movies_y'] = merge_2['common_movies_y'].fillna(0)
    merge_2['common_movies'] = merge_2['common_movies_x'] + merge_2['common_movies_y']

    return merge_2['rs_score'].corr(merge_2['common_movies']) # Your Answer Here

answer_four()

0.78839622217334748